In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

torch.cuda.empty_cache()

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2").cuda()

/home/s2320415/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
from StaICC.prefabricate_inference import model_kernel
import functools
from StaICC.prefabricate_inference import standard_calibration

oriinference = functools.partial(model_kernel.standard_ICL_inference_with_torch_Causal_LM, model = model, tokenizer = tokenizer, cache_empty = torch.cuda.empty_cache, return_hidden_state = True, return_full_vocab_prob = False) 
calibration = standard_calibration.hidden_calibration

In [5]:
from StaICC import Normal

benchmark = Normal()

Loading data...

1 in 10 Data loaded:  GLUE-SST2 

2 in 10 Data loaded:  rotten_tomatoes 

3 in 10 Data loaded:  financial_phrasebank 

4 in 10 Data loaded:  SST5 

5 in 10 Data loaded:  TREC 

6 in 10 Data loaded:  AGNews 

7 in 10 Data loaded:  Subjective 

8 in 10 Data loaded:  tweet_eval_emotion 

9 in 10 Data loaded:  tweet_eval_hate 

10 in 10 Data loaded:  hate_speech_18 

Data loaded successfully.

Initializing experimentor on k = 4...

Ready.



In [6]:
cali = []
for i in range(len(benchmark.experimentor)):
    print(i)
    tempcali = calibration(benchmark.experimentor[i].get_label_space())
    tempcali.train(benchmark.experimentor[i].get_prompt_writter_from_dataline(), oriinference, benchmark.experimentor[i].calibration_set(), calibration_number = 128)
    cali.append(tempcali)
    
inference = []
for i in range(len(benchmark.experimentor)):
    inference.append(functools.partial(model_kernel.standard_ICL_inference_with_torch_Causal_LM, model = model, tokenizer = tokenizer, cache_empty = torch.cuda.empty_cache, calibration_function = cali[i]))

0
Process: 100%, 128 in 128 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Calibration Training Finished.

1
Process: 100%, 128 in 128 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Calibration Training Finished.

2
Process: 100%, 128 in 128 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Calibration Training Finished.

3
Process: 100%, 128 in 128 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Calibration Training Finished.

4
Process: 100%, 128 in 128 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Calibration Training Finished.

5
Process: 100%, 128 in 128 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Calibration Training Finished.

6
Process: 100%, 128 in 128 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Calibration Training Finished.

7
Process: 100%, 128 in 128 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Calibration Training Finished.



/home/s2320415/StaICC_Expe/StaICC/prefabricate_inference/standard_calibration.py:172: UserWarning: Some categories didn't present in the calibration set: hate_speech_18-calibration
  warnings.warn("Some categories didn't present in the calibration set: " + str(calibration_set.get_dataset_name()))


In [9]:
output_dict = benchmark(inference)



Experiment 1 in 10

Start testing the forward inference function functools.partial(<function standard_ICL_inference_with_torch_Causal_LM at 0x14ad5c9c87c0>, model=GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): L

In [10]:
output_dict

{'Divided results': {'GLUE-SST2': {'accuracy': 0.5048828125,
   'averaged_truelabel_likelihood': 0.5013245637613665,
   'macro_F1': 0.5046329029137052,
   'expected_calibration_error_1': 0.3197234956710516},
  'rotten_tomatoes': {'accuracy': 0.5244140625,
   'averaged_truelabel_likelihood': 0.5176262426345551,
   'macro_F1': 0.52365041384284,
   'expected_calibration_error_1': 0.39269467683968473},
  'financial_phrasebank': {'accuracy': 0.47265625,
   'averaged_truelabel_likelihood': 0.47562066489317695,
   'macro_F1': 0.35554833377312245,
   'expected_calibration_error_1': 0.3500322328546429},
  'SST5': {'accuracy': 0.20703125,
   'averaged_truelabel_likelihood': 0.1949255241884676,
   'macro_F1': 0.18123741430123225,
   'expected_calibration_error_1': 0.2560959902764165},
  'TREC': {'accuracy': 0.2900390625,
   'averaged_truelabel_likelihood': 0.2621657558748818,
   'macro_F1': 0.2465559448620819,
   'expected_calibration_error_1': 0.41508126881429863},
  'AGNews': {'accuracy': 0.344